In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
# Obtiene gif de la solución
def get_animation(Q, x, U, Nt, title, titles, path):
    
    # Crea figura
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    ax1, ax2 = axs
    
    # Parámetros
    plt.xlabel(r'$x$')
    plt.ylabel(r'$Q(t, x)$')
    ax1.set_title(titles[0])
    ax2.set_title(title[1])
    
    fig.suptitle(title)
    
    ax1.set_ylim(-0.1, 0.5)
    ax2.set_ylim(-1, 60)
    
    ax1.set_xlabel("x")
    ax2.set_ylabel("y")
    
    # Valores iniciales
    rho_0 = Q[0][0]
    y_0 = Q[0][1]
    u_0 = u(rho_0, y_0, U)

    # Lineas iniciales
    line1, = ax1.plot(x, rho_0, '-', color='r', linewidth=2)
    line2, = ax2.plot(x, u_0, '-', color='b', linewidth=2, label=r"u")
    
    plt.legend()

    def animate(n):
        
        # Instante n
        rho = Q[n][0]
        y = Q[n][1]
        u_ = u(rho, y, U)
        
        # Actualiza gráfico
        line1.set_ydata(rho)
        line2.set_ydata(u_)

        ax1.set_title(titles[0])
        ax2.set_title(titles[1])
        
        return [line1, line2]

    ani = animation.FuncAnimation(fig, animate, np.arange(0, Nt, 3),
                                interval=50, blit=True)

    ani.save(path)

# Aplicación a ecuación ARZ

En primer lugar consideraremos el modelo ARZ homogeneo, que considera conservación de masa y del campo de velocidades shifteado. Las ecuaciones corresponden a
$$
\begin{align*}
\rho_t + (\rho u)_x &= 0\\
(u - U(\rho))_t + u(u-U(\rho))_x &= 0
\end{align*}
$$

donde $\rho = \rho(x, t)$ corresponde a la densidad vehicular, $u=u(x, t)$ al campo de velocidades y $U(\rho)$ velocidad de equilibrio. Consideraremos $U(\rho) = u_{\text{max}}(1-\rho^\gamma)$ con $\gamma \in (0, 1)$. En particular, se usarán $u_{\text{max}}=20$ y $\gamma=1/2$.

Definiendo $y(\rho, u)= \rho(u-U(\rho))$ y sumando ambas ecuaciones, podemos escribir el sistema en su forma conservativa:
$$
 \textbf{Q}_t + F( \textbf{Q})_x = 0
$$
donde $ \textbf{Q} = \begin{pmatrix} \rho \\ y \end{pmatrix}$ y $F(\textbf{Q}) = \begin{pmatrix} \rho u\\ yu\end{pmatrix}$. De esta forma se puede utilizar cualquier esquema de volúmenes finitos que se haya visto. En este caso, se usará el esquema de Godunov, que pide resolver el problema de Riemann asociado al modelo. Este se resolverá analiticamente y se programará separando en casos.

Una opción a manejar es resolver el problema de Riemann usando solvers aproximadores, pues se puede complejizar más si se resuelve el problema de Riemann no homogeneo. Se puede también resolver analíticamente pero resulta en un mayor coste computacional.

## Funciones del modelo

In [3]:
u_max = 40
gamm = 1/2

# Define u en función de rho e y
def u(rho, y, U):
    output = y/rho + U(rho)
    return output

# Define y en función de rho y u
def y_u(rho, u, U):
    output = rho*(u - U(rho))
    return output

In [4]:
# Flujo del modelo
def flux(Q, U):
    
    # Rescata variables
    rho, y = Q
    
    # Obtiene u en funcion de rho e y
    u_ = u(rho, y, U)
    return np.array([rho * u_, y * u_])

In [5]:
# Flujo de Godunov de primer orden
def F(Q, N, U, l):
    
    # Guarda flujo en un arreglo
    F_ = np.zeros(Q.shape)
    
    for i in range(1, N-1):
        
        # Rescata actual y vecinos
        Q_left = Q[:, i-1]
        Q_i = Q[:, i]
        Q_right = Q[:, i+1]
        
        # Problema de Riemann en cada vecino
        w_left = w(Q_left, Q_i, U)
        w_right = w(Q_i, Q_right, U)
        
        # Evalúa en el flujo del modelo
        F_[:, i] = flux(w_right, U) - flux(w_left, U)
    
    return F_

## Resuelve el problema de Riemann

In [6]:
# Solución problema de Riemann homogeneo

# TODO: Quitar casos redundantes
# TODO: Cambiar si se quiere cambiar el h(rho) o U(rho)
# TODO: Agregar Riemann no homogeneo
def w(Q_l, Q_r, U, umax=u_max, gamma=gamm):
    
    # Rescata variables a la izquierda
    rho_l, y_l = Q_l
    
    # Rescata variables a la derecha
    rho_r, y_r = Q_r
      
    # Obtiene velocidades en caso que densidades son positivas
    u_l = u(rho_l, y_l, U)
    u_r = u(rho_r, y_r, U)
    u_m = u_r

    # Define valor medio de rho y u
    rho_m = (rho_l**gamma + (u_l - u_r)/umax)**(1/gamma)

    # u_r y u_l cercanos
    if np.isclose(u_r, u_l):

        # Solucion en 0
        rho_0 = rho_l
        u_0 = u_l

    # u_r menor a u_l
    elif u_r < u_l:

        # Define velocidad del shock
        l_s = (rho_m * u_m - rho_l * u_l)/(rho_m - rho_l)

        # Si es positivo
        if l_s >= 0:

            # Solucion en 0
            rho_0 = rho_l
            u_0 = u_l

        # Si es negativo
        else:

            # Solucion en 0
            rho_0 = rho_m
            u_0 = u_m

    # Condición u_l en un intervalo en función de u_r
    elif u_r - umax*rho_l**gamma < u_l < u_r:

        # Lambdas a la izquierda y centrado con respecto al 0
        l_0_l = u_l - umax * gamma * rho_l**gamma
        l_0_m = u_r - umax*gamma*rho_l**gamma + gamma*(u_r - u_l)

        # Lambda izquierda es positivo
        if l_0_l >= 0:

            # Solucion en 0
            rho_0 = rho_l
            u_0 = u_l

        # Lambda derecha es negativo
        elif l_0_m <= 0:

            # Solucion en 0
            rho_0 = rho_m
            u_0 = u_m

        # A la izquierda es negativo y centrado positivo
        elif l_0_l < 0 < l_0_m:
            rho_bar = ((u_l + umax*rho_l**gamma)/((gamma + 1)*umax))**(1/gamma)
            u_bar = (gamma/(gamma+1)) * (u_l + umax*rho_l**gamma)

            # Solucion en 0
            rho_0 = rho_bar
            u_0 = u_bar

    # u_l menor a u_r menos algo
    elif u_l <= u_r - umax*rho_l**gamma:

        # Lambda a la izquierda del 0
        l_0_l = u_l - umax * gamma * rho_l**gamma

        # Si es positivo
        if l_0_l >= 0:

            # Solucion en 0
            rho_0 = rho_l
            u_0 = u_l

        # Si es negativo
        else:
            rho_bar = ((u_l + umax*rho_l**gamma)/((gamma + 1)*umax))**(1/gamma)
            u_bar = (gamma/(gamma+1)) * (u_l + umax*rho_l**gamma)

            # Solucion en 0
            rho_0 = rho_bar
            u_0 = u_bar

    # rho_l cero y rho_r positivo
    # TODO: Ver equivalencia a algun caso anterior
    elif np.isclose(rho_l, 0) and rho_r >0:
        rho_0 = 0
        u_0 = 0
    
    # Opuesto al anterior
    # TODO: Ver equivalencia a algun caso anterior
    elif rho_l > 0 and np.isclose(rho_r, 0):
        
        # Velocidad
        u_l = u(rho_l, y_l, U)
        
        # Obtiene lambda a la izquierda del 0
        l_0_l = u_l - umax * gamma * rho_l**gamma
        
        # Si es positivo
        if l_0_l >= 0:
            
            # Solucion en 0
            rho_0 = rho_l
            u_0 = u_l
        
        # Si es negativo
        else:
            rho_bar = ((u_l + umax*rho_l**gamma)/((gamma + 1)*umax))**(1/gamma)
            u_bar = (gamma/(gamma+1)) * (u_l + umax*rho_l**gamma)
            
            # Solucion en 0
            rho_0 = rho_bar
            u_0 = u_bar
    
    # Condición auxiliar por si no se cumplen las anteriores
    else:
        rho_0 = 0
        u_0 = 0
        
    
    # Obtiene y_0 segun el u_0 obtenido
    y_0 = y_u(rho_0, u_0, U)
    
    return np.array([rho_0, y_0])
